In [1]:
import cvxpy as cp
import numpy as np

### Objective Function : 
$$\left\{\sum_{t=1}^{T}{|e\left(t\right)-e\left(t-1\right)}|\right\}+\left(1-\frac{L^{Avg}}{L^{Max}}\right)+{dem}^{peak}+e^{net}+emi$$

- $ e(t) = E_t^{NS} + E_t^{bd\_C} + E_t^{bd\_DHW} + E_t^{bat} - E_t^{PV}$ <!-- E_t^{PV} : param -->
<!-- et =  - E_t^{grid} -->
<br>

- $ L^{Avg} = \frac{1}{24}\sum_{\tau = t}^{t + 24} e(\tau) $ <!-- switched to per day, instead of per month -->
<br>

- $ L^{Max} = \smash{\displaystyle\max_{1 \leq \tau \leq T}} e(\tau)$
<br>

- $dem^{peak} = \smash{\displaystyle\max_{\tau \in T}} \> e(\tau)$
<br>

- $e^{net} = \sum_{t = 1}^T \> e(t) $ <!-- clip to 0 i.e, min(0, e(t)) -->
<br>

- $emi_t = \sum_{t = 1}^T  e(t) \times CI_t $

> treat each day as an episode. 

> L_max => calculate w/ $E^{grid} $. 

## Constraints:

$$ e^{net} \geq 0 $$

$$ E^{NS} + E^{bd\_C} + E^{bd\_DHW} + E^{bat} == E^{PV} $$

- $ E^{bd\_C} = E^{hp\_C} = \frac{C^{hp}}{COP^C} $
<br>

- $ E^{bd\_DHW} = E^{eh\_H} = \frac{H^{eh}}{\text{efficiency}^h} $
<br>

- $ E^{bat} = charge(\text{action}\times \text{capacity}) $
- $ \text{action} \in [-1, 1] \in ℝ $
- $ \text{SOC}_t = \text{SOC}_{t-1}(1 - loss_{coef}) + \text{action} \times \text{capacity}$
- $ charge = \text{SOC}_t - \text{SOC}_{t-1}(1 - loss_{coef}) $ <!-- Solve for action -->

<!-- capacity needs to be updated every timestep: https://github.com/QasimWani/CityLearn/blob/ffc0584508dc9c796c97e6b908b75380b9bc6606/energy_models.py#L731 -->

<br>

- $ E^{grid} = \text{nominal power} $  <!-- nominal power is param -->
<br>

$$  H^{eh} == H^{STO} + H^{bd} $$
<!-- All 3 params -->

<!-- H^bd is param -->
<!-- H^STO depends on action (https://github.com/QasimWani/CityLearn/blob/ffc0584508dc9c796c97e6b908b75380b9bc6606/energy_models.py#L123) -->

- $ H^{STO} = \text{depends on action} $

<!-- $ H^{hp} = E^{hp\_h} + COP^H $ treat H^hp as param. don't define it in terms of LHS -->
<!-- H^eh is param (defined https://github.com/QasimWani/CityLearn/blob/ffc0584508dc9c796c97e6b908b75380b9bc6606/energy_models.py#L515) -->

$$ C^{hp} == C^{bd} + C_{toSTO}^{dev} - C^{STO} $$

<!-- C^bd is param -->
- $ C^{dev} = C^{hp}  $ <!-- C^hp is param -->
<br>

- $ C^{STO} = \text{depends on action} $ <!-- treat as param -->
<!-- How is C_{toSTO} different from C^{sto} -->

### Variables

<ol>
    <li>$ E^{bat}$ </li>
    <li>$ H^{STO}$ </li>
    <li>$ C^{STO}$ </li>
</ol>

### Optimization

In [2]:
get_data = lambda x : np.random.rand(x) #returns random data of shape `x`

In [3]:
e_tau = lambda a, b, c, d, e : a + b + c + d - e #follow order from Obj. function (above)

In [4]:
#define parameters and variables
constraints = []

#define timesteps
T = 24#*365*4
window = 24

#define misc. params 
CI = cp.Parameter(name='CI', shape=(T), value=get_data(T)) #carbon intensity

#heating
H_sto = cp.Variable(name='H^STO', shape=(T)) 
H_bd = cp.Parameter(name='H^bd', shape=(T), value=get_data(T))
H_eh = cp.Parameter(name='H^eh', shape=(T), value=get_data(T))
efficiency_h = cp.Parameter(name='eff^h', shape=(1), value=get_data(1)) #every building has different value

constraints.append(H_eh == H_sto + H_bd) #H_eh = H_sto + H_bd

#cooling
C_sto = cp.Variable(name='C^STO', shape=(T))
C_bd = cp.Parameter(name='C^bd', shape=(T), value=get_data(T))
C_dev = cp.Parameter(name='C^dev_toStorage', shape=(T), value=get_data(T))
COP_C = cp.Parameter(name='COP^C', shape=(T), value=get_data(T))
C_hp = cp.Parameter(name='C^hp', shape=(T), value=get_data(T))

constraints.append(C_hp == C_bd + C_dev - C_sto) #C_hp = C_bd + C_dev - C_sto

#electricity
E_bat = cp.Variable(name='E^bat', shape=(T))
E_ns = cp.Parameter(name='E^NS', shape=(T), value=get_data(T))
E_bd_C =  C_hp / COP_C
E_bd_dhw = H_eh / efficiency_h
E_pv = cp.Parameter(name='E^pv', shape=(T), value=get_data(T))

constraints.append(E_ns + E_bd_C + E_bd_dhw + E_bat == E_pv )

In [5]:
#define objective function
costs = []

ramping = []
for i in range(1, T):
    e_t0 = e_tau( E_ns[i], E_bd_C[i], E_bd_dhw[i], E_bat[i], E_pv[i] )
    e_t1 = e_tau( E_ns[i - 1], E_bd_C[i - 1], E_bd_dhw[i - 1], E_bat[i - 1], E_pv[i - 1] )
    ramping.append( cp.abs(e_t0 - e_t1) )
    
ramping = cp.atoms.affine.add_expr.AddExpression(np.hstack(ramping))

# 1-load_factor
l_avg = cp.sum( e_tau(E_ns[:window], E_bd_C[:window], E_bd_dhw[:window], 0, E_pv[:window]) ) / window
l_max = cp.atoms.maximum( *e_tau(E_ns, E_bd_C, E_bd_dhw, E_bat, E_pv) )

load_factor = 1 + l_max / l_avg.value

dem_peak = l_max #replace with overall time horizon (4yrs) -- need confirmation on functionality

e_net = e_tau(E_ns, E_bd_C, E_bd_dhw, E_bat, E_pv)

emi = cp.atoms.affine.binary_operators.MulExpression(e_net, CI)

constraints.append(e_net >= 0)

e_net = cp.sum(e_net)


costs.append( ramping + load_factor + dem_peak + e_net + emi )

In [6]:
# Form objective.
obj = cp.Minimize(*costs)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.

/Users/qasimwani/opt/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


1.0000000000556095

In [7]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var:")
for var in prob.variables():
    print(var.name(), np.round(var.value, 2), '\n')

status: optimal
optimal value 1.0000000000556095
optimal var:
E^bat [ -3.66  -0.57  -1.96  -2.2  -10.35  -3.39  -9.24  -1.06  -3.17  -2.78
  -1.28  -4.44  -3.22  -3.82  -1.39  -2.33  -5.58  -3.83  -2.7   -5.5
  -2.76  -2.68 -63.71  -4.29] 

H^STO [-0.21 -0.65  0.1   0.11 -0.45  0.48  0.49 -0.38  0.07  0.16 -0.34  0.3
  0.17  0.39 -0.64 -0.09 -0.33  0.02  0.16  0.17  0.05 -0.44 -0.33  0.73] 

C^STO [-0.01  0.71  0.43  1.13  0.42  0.78 -0.42  1.41  0.81  0.71  1.36  0.57
  0.66 -0.01  0.25  0.17  0.09 -0.16  0.16  0.96  0.18  0.6   0.39  0.96] 



### Convert vars into action for cooling, heating, and storage

In [8]:
# class MyProblem(self, n, ...):
#     self._var = Variable(n)
#     self.param = Parameter(n)

#     costs = #Some complex convex function of var and param
#     self._problem = Problem(Minimize(costs), constraints)

#     def solve(self):
#         self._problem.solve()
#         return self._target

# problem = MyProblem(4, ...)
# for param_value in param_values:
#     problem.param.value = param_value
#     answer = problem.solve()